In [1]:
import pickle
import shap
import numpy as np
import lightgbm 
from rdkit.Chem import PandasTools, AllChem, rdFingerprintGenerator
from pandas import read_csv, DataFrame
from random import seed
import matplotlib.pyplot as plt

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
data = read_csv('../../../DATABASE/no_missing_data.csv')
data

,molecule,smiles,max_abs,max_em,solvent,dc,etn
0,BTD-QN,c4cnc3c(Nc1cccc2nsnc12)cccc3c4,445.0,620.0,DMSO,46.8260,0.444
1,BTD-QN,c4cnc3c(Nc1cccc2nsnc12)cccc3c4,445.0,565.0,H2O,78.3553,1.000
2,BTD-QN,c4cnc3c(Nc1cccc2nsnc12)cccc3c4,440.0,575.0,AcOEt,6.0200,0.228
3,BTD-QN,c4cnc3c(Nc1cccc2nsnc12)cccc3c4,450.0,615.0,DCM,8.9300,0.309
4,BTD-QN,c4cnc3c(Nc1cccc2nsnc12)cccc3c4,440.0,615.0,ACN,35.6880,0.460
...,...,...,...,...,...,...,...
696,Priscila-5,C(C#Cc1ccc(C#CCN(c2ccccc2)c3ccccc3)c4nsnc14)N(...,380.0,536.0,DCM,8.9300,0.309
697,Priscila-5,C(C#Cc1ccc(C#CCN(c2ccccc2)c3ccccc3)c4nsnc14)N(...,374.0,549.0,ACN,37.5000,0.460
698,Priscila-7,C(C#Cc1ccc(C#CCN(c2ccccc2)c3ccccc3)c4nsnc14)N5...,376.0,570.0,DIOX,2.2500,0.164
699,Priscila-7,C(C#Cc1ccc(C#CCN(c2ccccc2)c3ccccc3)c4nsnc14)N5...,377.0,560.0,DCM,8.9300,0.309


In [3]:
seed(0)

np.random.seed(0)


PandasTools.AddMoleculeColumnToFrame(data, 'smiles', 'ROMol')
        # Create the fingerprint generator
fp_gen = rdFingerprintGenerator.GetMorganGenerator(radius = 4, fpSize = 2048)

# Generate fingerprints for each molecule
morgan_fps = [list(fp_gen.GetFingerprint(m)) for m in data['ROMol']]

morgan_df = DataFrame(morgan_fps)


morgan_df.columns = ['MF_' + str(j) for j in morgan_df.columns]
morgan_df['etn'] = data['etn'].to_numpy()
# morgan['dc'] = data['dc'].values
morgan_df

,MF_0,MF_1,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_8,MF_9,...,MF_2039,MF_2040,MF_2041,MF_2042,MF_2043,MF_2044,MF_2045,MF_2046,MF_2047,etn
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.444
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.000
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.228
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.309
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.309
697,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.460
698,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.164
699,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.309


In [4]:
seed(0)
em_model = pickle.load(open('rf_kf_max_em.pkl', 'rb'))
abs_model = pickle.load(open('rf_kf_max_abs.pkl', 'rb'))


In [7]:
seed(0)
explainer_em = shap.TreeExplainer(em_model)
explainer_abs = shap.TreeExplainer(abs_model)


shap_values_em = explainer_em.shap_values(morgan_df.to_numpy())
shap_values_abs = explainer_abs.shap_values(morgan_df.to_numpy())


In [16]:
# Create SHAP Explanation object
shap_values_em_exp = shap.Explanation(shap_values_em, data=morgan_df)
shap_values_em_exp.feature_names = morgan_df.columns

# Generate the SHAP bar plot
shap.plots.bar(shap_values_em_exp, max_display=15, show=False)

# Save the figure
plt.savefig("shap_importance_em.png", dpi=300, bbox_inches='tight')

# Close the plot
plt.close()


In [17]:
# Create SHAP Explanation object
shap_values_abs_exp = shap.Explanation(shap_values_abs, data=morgan_df)
shap_values_abs_exp.feature_names = morgan_df.columns

# Generate the SHAP bar plot
shap.plots.bar(shap_values_abs_exp, max_display=15, show=False)

# Save the figure
plt.savefig("shap_importance_abs.png", dpi=300, bbox_inches='tight')

# Close the plot
plt.close()

In [ ]:
shap.summary_plot(shap_values_abs, morgan_df, show=False)
#save the plot
import matplotlib.pyplot as plt
plt.savefig('shap_rf_abs.png')
plt.close()

In [ ]:
shap.summary_plot(shap_values_em, morgan_df, show=False)

In [ ]:
shap.summary_plot(shap_values_abs, morgan_df, show=False)

In [ ]:
vals = np.abs(shap_values_abs).mean(0)

shap_importance = DataFrame(list(zip(morgan_df.columns, vals)), columns=['col_name', 'feature_importance_vals'])

top10_morgan = shap_importance.nlargest(12, 'feature_importance_vals')

top10_morgan = top10_morgan.drop([2048])

top10_morgan

In [ ]:
from drawSmiles import *
import cairosvg

NamesMol = data['molecule']
SolventMol = data['solvent']

best_radius = 4
best_length = 2048

IPythonConsole.ipython_useSVG = True 

bits = [int(x.split('_')[1]) for x in top10_morgan['col_name'] if x.startswith('MF_')]

for j in bits:
    bit_number = j
    molNumber = morgan_df.loc[morgan_df['MF_{}'.format(bit_number)] == 1].index[0]
        
    smiles = data['smiles'].iloc[molNumber]

    mol = Chem.MolFromSmiles(smiles)
    
    addcords(mol)
    info = {}

    fp = AllChem.GetMorganFingerprintAsBitVect(mol,radius=best_radius,bitInfo=info,nBits=best_length)

    X = draw_molecule_and_bit_info_grid(mol,info,bit_number, 400)
    cairosvg.svg2png(X.data, write_to='Morgan_Fingerprint_{}.png'.format(bit_number))
  
    print('Molecule_{}'.format(NamesMol[molNumber]))
    print('Morgan Fingerprint: {}'.format(bit_number))
    print('Smiles: {}'.format(data['smiles'].iloc[molNumber]))  
    display(X)
    #save the image